In [36]:
import os 
import json
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [28]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    if api_key.startswith('sk-proj'):
        print("GPT api key exist")
    else:
        print("GPT api key does not exist")
else:
    print("no api key")

GPT api key exist


In [29]:
openai = OpenAI()

In [30]:

system_prompt = "You are a helpful AI assistant"

def chat_gpt(message,history):

    messages = [{'role':'system','content':system_prompt}] + history + [{'role':'user','content':message}]

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )

    return response.choices[0].message.content

In [32]:
gr.ChatInterface(fn=chat_gpt,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [44]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [45]:
# a particular dir structure require by ChatGPT

price_function = {
    "name":"get_ticket_price",
    "description": "get the price of ticket to destination city use this when you are asked the ticket price for a city" + "for example when a customer asks : 'How much is the ticket price to this city'",
    "parameters": 
    {
        "type": "object",
        "properties" : 
        {
            "destination_city":
            {
                "type":"string",
                "description":"The city that the customer wants to travel to",
            }
        },
        "required" : ["destination city"],
        "additionalProperties" : False
    }
}

In [46]:
# There will also be list of tools

tools = [{"type":"function","function":price_function}]

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [60]:
def chat(user_msg,history):

    messages = [{'role':'system','content':system_message}] + history + [{'role':'user','content':user_msg}]

    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)

    # return response.choices[0].message.content  // Now we don't return answer we check if model wants to call tool for answer 
    print("returned output 1 : ",response.choices[0])

    # working --> user ask que to model --> assistant msg : ( it says it rquires to call tools to answer question ) --> tool gets called --
    # --> result comes back --> we again pass msg to model which is combination of (assistant msg + tool result ) --> chatgpt generates formatted output 


    if response.choices[0].finish_reason == "tool_calls": # checking if model asks for tool calls 
        
        message = response.choices[0].message 
        
        print("message asked during toolcall:",message)
        
        response , city = handle_tool_call(message)
        
        messages.append(message)
        messages.append(response)

        print("response fromm tool",response)

        response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)

    return response.choices[0].message.content
    

In [61]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0] # extracting tool name model wanted to call 

    print("Tool named model asked for : ",tool_call)

    arguments = json.loads(tool_call.function.arguments)  # convert string to json
    city = arguments.get('destination_city') # extracting destination city from json 
    price = get_ticket_price(city) # calling our function 
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}), # converts json to string 
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [63]:
gr.ChatInterface(fn=chat, type="messages").launch(debug=True)

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


returned output 1 :  Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))
returned output 1 :  Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vKI9rm9YeKiTLQ7eNti4Qp27', function=Function(arguments='{"destination_city":"Tokyo"}', name='get_ticket_price'), type='function')]))
message asked during toolcall: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vKI9rm9YeKiTLQ7eNti4Qp27', function=Function(arguments='{"destination_city":"Tokyo"}', name='get_ticket_price'), type='function')])
Tool named model asked for :  ChatCompletionMessageToolCall(id='call_vKI9rm9YeKiTL